In [76]:
import pandas as pd
import numpy as np
obj = {}

df = pd.read_csv('./tweets.csv')
obj['text'] = df['text']
dataframe = pd.DataFrame(obj)
dataframe.head(10)

/home/buckaroo/miniconda3/envs/dev/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (27,194,197) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,text
0,RT @KirkKus: Indirect cost of the UK being in ...
1,VIDEO: Sturgeon on post-election deals http://...
2,RT @LabourEoin: The economy was growing 3 time...
3,RT @GregLauder: the UKIP east lothian candidat...
4,RT @thesundaypeople: UKIP's housing spokesman ...
5,RT @Nigel_Farage: Make sure you tune in to #As...
6,RT @joannetallis: Ed Milliband is an embarrass...
7,RT @abstex: The FT is backing the Tories. On a...
8,RT @NivenJ1: “@George_Osborne: Ed Miliband pro...
9,LOLZ to Trickle Down Wealth. It's never trickl...


In [36]:
print(dataframe.shape)

(20000, 1)


In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(
    max_df=.5,
    min_df=10,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)
vectorizer = vectorizer.fit(dataframe['text'])
tfidf = vectorizer.transform(dataframe['text'])
vocab = vectorizer.get_feature_names()
print(vocab[0])
print(len(vocab))

000
9006


In [38]:
from corextopic import corextopic as ct

In [39]:
anchors = []
model = ct.Corex(n_hidden=4, seed=42)
model = model.fit(
    tfidf,
    words=vocab
)

In [40]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: rt nicolasturgeon, nicolasturgeon if, will definitely, we will, definitely need, protect sco, is going, need lots, in rather, snp we
Topic #2: tories claiming, claiming, preoccupied, is preoccupied, in support, preoccupied inequality, inequality the, wrote it, who wrote, claiming miliband
Topic #3: says he, are all, never forgive, says, you hear, off the, forgive, into, off, into new
Topic #4: snp, with, rather, with snp, deal, let, work with, work, deal with, tories in


In [41]:
# Anchors designed to nudge the model towards measuring specific genres
anchors = [
    ["money","fund"],
    ["emergency"],
    ["recovered"],
    ["treatment"]
]
anchors = [
    [a for a in topic if a in vocab]
    for topic in anchors
]

model = ct.Corex(n_hidden=4, seed=42)
model = model.fit(
    tfidf,
    words=vocab,
    anchors=anchors, # Pass the anchors in here
    anchor_strength=3 # Tell the model how much it should rely on the anchors
)

In [42]:
for i, topic_ngrams in enumerate(model.get_topics(n_words=10)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("Topic #{}: {}".format(i+1, ", ".join(topic_ngrams)))

Topic #1: is preoccupied, tommy_colc, preoccupied, in support, who wrote, preoccupied inequality, rt tommy_colc, inequality the, wrote it, claiming miliband
Topic #2: rather, with, let, than work, work with, than, work, let tories, tories in, with snp
Topic #3: says he, global financial, caused global, thus caused, thomasmessenger for, caused, and thus, ahem http, crisis ahem, thomasmessenger
Topic #4: http, http co, co, financial, bbcqt http, david cameron, david, cameron, farage, to


In [43]:
def arr_to_topic(arr=[]):
    array = ""
    for s in arr:
        array+=s+" "
        break
    
    return array   

In [44]:
topic_df = pd.DataFrame(
    model.transform(tfidf), 
    #columns=["topic_{}".format(i+1) for i in range(4)]
    columns=  ["money","emergency","recovered","treatment"]
).astype(float)


topic_df.index = dataframe.index

print(dataframe.shape)
print(topic_df.shape)

def filter(x):
    columns = ["money","emergency","recovered","treatment"]
    str = []
    for col in columns:
        if bool(x[col]):
            str.append(col)
    return str

obj= {}

obj["topics"] = topic_df.apply( filter , axis=1 )

obj =  pd.DataFrame(obj)

final = pd.concat([dataframe, obj], axis=1)

(20000, 1)
(20000, 4)


In [45]:
final.sample(100, random_state=92)


,text,topics
15119,RT @STUD6699: Nigel Farage says that homosexua...,[recovered]
12499,RT @UKIPEaling: Nigel Farage speaking with con...,[]
10326,@ukmarkus http://t.co/WkfVKwHgED,[treatment]
238,RT @A_Liberty_Rebel: Only regret at #BBCQT is ...,[]
1943,#bbcqt I'm surprised Cameron only got 44%. The...,[treatment]
...,...,...
7762,RT @UKIP: 70% of Britons support an Australian...,[treatment]
15774,RT @ad5red: @Nigel_Farage Different class toni...,[]
14655,RT @NicolaSturgeon: If Miliband is going to le...,[emergency]
16025,Spot on Alison Parr; he did a fine job; unruff...,[]


In [46]:
df.sample(10,random_state=43)


,_id,contributors,coordinates,created_at,entities.hashtags.0.indices.0,entities.hashtags.0.indices.1,entities.hashtags.0.text,entities.hashtags.1.indices.0,entities.hashtags.1.indices.1,entities.hashtags.1.text,...,user.profile_sidebar_fill_color,user.profile_text_color,user.profile_use_background_image,user.protected,user.screen_name,user.statuses_count,user.time_zone,user.url,user.utc_offset,user.verified
5393,5e9f192a8b39a129cc25d37d,NaN,NaN,Thu Apr 30 21:53:09 +0000 2015,41.0,54.0,OffshoreDave,NaN,NaN,NaN,...,DDEEF6,333333,True,False,Marcusdc77,1760,London,NaN,3600.0,False
15408,5e9f192c8b39a129cc25fa9c,NaN,NaN,Thu Apr 30 22:28:12 +0000 2015,NaN,NaN,NaN,NaN,NaN,NaN,...,DDEEF6,333333,True,False,scotexpress46,5814,NaN,NaN,NaN,False
6799,5e9f192b8b39a129cc25d8fb,NaN,NaN,Thu Apr 30 21:58:03 +0000 2015,139.0,140.0,GE15,139.0,140.0,voteSNP,...,DDEEF6,333333,True,False,Jacqwalsh,9052,NaN,NaN,NaN,False
2773,5e9f192a8b39a129cc25c941,NaN,NaN,Thu Apr 30 21:43:45 +0000 2015,NaN,NaN,NaN,NaN,NaN,NaN,...,C0DFEC,333333,True,False,McPetey94,7649,Edinburgh,NaN,3600.0,False
1877,5e9f19298b39a129cc25c5c1,NaN,NaN,Thu Apr 30 21:40:25 +0000 2015,127.0,133.0,BBCqt,NaN,NaN,NaN,...,DDEEF6,333333,True,False,CRoseTweets,452,NaN,NaN,NaN,False
4649,5e9f192a8b39a129cc25d095,NaN,NaN,Thu Apr 30 21:50:54 +0000 2015,NaN,NaN,NaN,NaN,NaN,NaN,...,DDEEF6,333333,True,False,meljomur,49114,Amsterdam,http://meljomur.wordpress.com,7200.0,False
16062,5e9f192c8b39a129cc25fd2a,NaN,NaN,Thu Apr 30 22:30:32 +0000 2015,40.0,53.0,Nigel_Farage,134.0,139.0,UKIP,...,DDEEF6,333333,True,False,Yorkshire_Hero,6688,London,http://www.bluesideup.co.uk,3600.0,False
2247,5e9f192a8b39a129cc25c733,NaN,NaN,Thu Apr 30 21:41:47 +0000 2015,NaN,NaN,NaN,NaN,NaN,NaN,...,7AC3EE,3D1957,True,False,grancake,38484,London,NaN,3600.0,False
8244,5e9f192b8b39a129cc25dea0,NaN,NaN,Thu Apr 30 22:03:21 +0000 2015,NaN,NaN,NaN,NaN,NaN,NaN,...,DDEEF6,333333,True,False,nbjinkx,343,Dublin,http://eddiecatflap.flavors.me/,3600.0,False
12605,5e9f192c8b39a129cc25efa9,NaN,NaN,Thu Apr 30 22:18:28 +0000 2015,NaN,NaN,NaN,NaN,NaN,NaN,...,EFEFEF,333333,True,False,nauraavesiharma,10320,Dublin,NaN,3600.0,False


In [49]:
! curl -L -o result.json https://twitter-covid-app.herokuapp.com/api/corona/0/9999

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 49736  100 49736    0     0  17056      0  0:00:02  0:00:02 --:--:-- 17056


In [80]:
import json

tweetJSON = {}

with open('./result.json' , encoding='utf-8') as J:
    tweetJSON = json.loads(J.read())
    
tweets = json.loads(tweetJSON['response']['tweets'])

tweet_texts = []

for tweet in tweets:
    tweet_texts.append(tweet['text'])

In [81]:
vectorizer = TfidfVectorizer(
    max_df=1,
    min_df=1,
    max_features=None,
    ngram_range=(1, 2),
    norm=None,
    binary=True,
    use_idf=False,
    sublinear_tf=False
)

dataframe = pd.DataFrame({
    'text' : np.array(tweet['text'])
},
index=[0])

vectorizer = vectorizer.fit(dataframe['text'])

tfidf = vectorizer.transform(dataframe['text'])

input_text = vectorizer.get_feature_names()

input_text

['26',
 '26 on',
 '768',
 '768 yesterday',
 'cases',
 'cases reported',
 'corona',
 'corona cases',
 'day',
 'highest',
 'highest day',
 'in',
 'in was',
 'number',
 'number of',
 'of',
 'of corona',
 'olafgersemann',
 'olafgersemann the',
 'on',
 'on the',
 'plus',
 'plus 26',
 'previous',
 'previous week',
 'reported',
 'reported to',
 'rki',
 'rki in',
 'rt',
 'rt olafgersemann',
 'the',
 'the number',
 'the previous',
 'the rki',
 'to',
 'to the',
 'was',
 'was 768',
 'week',
 'week highest',
 'yesterday',
 'yesterday plus']

In [83]:
topics = model.get_topics()

In [84]:
topics

[[('is preoccupied', 0.0993932088537624, 1.0),
  ('tommy_colc', 0.0993932088537624, 1.0),
  ('preoccupied', 0.09938559344981493, 1.0),
  ('in support', 0.09922630730459366, 1.0),
  ('who wrote', 0.09905943968328636, 1.0),
  ('preoccupied inequality', 0.09905943968328636, 1.0),
  ('rt tommy_colc', 0.09905943968328636, 1.0),
  ('inequality the', 0.09905943968328636, 1.0),
  ('wrote it', 0.09905943968328636, 1.0),
  ('claiming miliband', 0.09905943968328636, 1.0)],
 [('rather', 0.16431801515920347, 1.0),
  ('with', 0.1274054054989736, 1.0),
  ('let', 0.1238232538210541, 1.0),
  ('than work', 0.11936320229010447, 1.0),
  ('work with', 0.11674752335118103, 1.0),
  ('than', 0.11448225521966718, 1.0),
  ('work', 0.09951239453515921, 1.0),
  ('let tories', 0.09314654124776815, 1.0),
  ('tories in', 0.09078722415976896, 1.0),
  ('with snp', 0.08894323171629916, 1.0)],
 [('says he', 0.026891434818868267, 1.0),
  ('global financial', 0.021059152413000107, 1.0),
  ('caused global', 0.0208998961492